In [2]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from transformers import SegformerFeatureExtractor
from transformers import Trainer, TrainingArguments
from transformers import SegformerForSemanticSegmentation
from dataset import CustomImageDataset

segmodel = SegformerForSemanticSegmentation.from_pretrained(
            "nvidia/segformer-b0-finetuned-ade-512-512", 
            return_dict=False, 
            num_labels=3,
            #id2label=self.id2label,
            #label2id=self.label2id,
            ignore_mismatched_sizes=True,
        )

# 모델 및 피쳐 추출기 로드
feature_extractor = SegformerFeatureExtractor.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")
feature_extractor.reduce_labels = False
feature_extractor.size = 128

local_data_path = 'C:\Workspace\Personal\ccp_data/Training/image/AP10_City_IMAGE'
server_data_path = '/mnt/nas27_dataset/Dataset/ccp/Training/image/AP10_City_IMAGE/'
colab_data_path = '/content/gdrive/MyDrive/ColabNotebooks/ViT/Training/image/SN10_Forest_IMAGE'

# Fine-tuning에 사용할 데이터셋 클래스 초기화
train_dataset = CustomImageDataset(folder_path=colab_data_path, transform=None, mode="Train")
valid_dataset = CustomImageDataset(folder_path=colab_data_path, transform=None, mode="Valid")

# 데이터 로더 설정
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=8, shuffle=False)

# 모델을 fine-tuning하기 위한 Trainer 및 TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./output",
    num_train_epochs=1,  # 1 epoch만 학습
    per_device_train_batch_size=8,
    save_total_limit=2,
    learning_rate=1e-4,
)

trainer = Trainer(
    model=segmodel,
    args=training_args,
    data_collator=None,  # 데이터셋에 따라 적절한 collate_fn 사용 (예: default_data_collator)
    train_dataset=train_dataloader,
)

# 학습 수행
trainer.train()

# 학습된 모델 저장
trainer.save_model("./fine_tuned_segformer")



C:\Workspace\Personal\ccp_data/Training/image/AP10_City_IMAGE Done.
C:\Workspace\Personal\ccp_data/Training/image/AP10_City_SH Done.
C:\Workspace\Personal\ccp_data/Training/label/AP10_City_Carbon Done.
C:\Workspace\Personal\ccp_data/Training/label/AP10_City_GT Done.
C:\Workspace\Personal\ccp_data/Validation/image/AP10_City_IMAGE Done.
C:\Workspace\Personal\ccp_data/Validation/image/AP10_City_SH Done.
C:\Workspace\Personal\ccp_data/Validation/label/AP10_City_Carbon Done.
C:\Workspace\Personal\ccp_data/Validation/label/AP10_City_GT Done.


In [5]:
import pytorch_lightning as pl
from transformers import SegformerForSemanticSegmentation
from datasets import load_metric

import torch
from torch import nn

import numpy as np

class SegformerFinetuner(pl.LightningModule):
    
    def __init__(self, id2label, train_dataloader=None, val_dataloader=None, test_dataloader=None, metrics_interval=100):
        super(SegformerFinetuner, self).__init__()
        self.id2label = id2label
        self.metrics_interval = metrics_interval
        self.train_dl = train_dataloader
        self.val_dl = val_dataloader
        self.test_dl = test_dataloader
        
        self.num_classes = len(id2label.keys())
        self.label2id = {v:k for k,v in self.id2label.items()}
        
        self.model = SegformerForSemanticSegmentation.from_pretrained(
            "nvidia/segformer-b0-finetuned-ade-512-512", 
            return_dict=False, 
            num_labels=self.num_classes,
            id2label=self.id2label,
            label2id=self.label2id,
            ignore_mismatched_sizes=True,
        )
        
        self.train_mean_iou = load_metric("mean_iou")
        self.val_mean_iou = load_metric("mean_iou")
        self.test_mean_iou = load_metric("mean_iou")
        
    def forward(self, images, masks):
        outputs = self.model(pixel_values=images, labels=masks)
        return(outputs)
    
    def training_step(self, batch, batch_nb):
        
        images, masks = batch['pixel_values'], batch['labels']
        
        outputs = self(images, masks)
        
        loss, logits = outputs[0], outputs[1]
        
        upsampled_logits = nn.functional.interpolate(
            logits, 
            size=masks.shape[-2:], 
            mode="bilinear", 
            align_corners=False
        )

        predicted = upsampled_logits.argmax(dim=1)

        self.train_mean_iou.add_batch(
            predictions=predicted.detach().cpu().numpy(), 
            references=masks.detach().cpu().numpy()
        )
        if batch_nb % self.metrics_interval == 0:

            metrics = self.train_mean_iou.compute(
                num_labels=self.num_classes, 
                ignore_index=255, 
                reduce_labels=False,
            )
            
            metrics = {'loss': loss, "mean_iou": metrics["mean_iou"], "mean_accuracy": metrics["mean_accuracy"]}
            
            for k,v in metrics.items():
                self.log(k,v)
            
            return(metrics)
        else:
            return({'loss': loss})
    
    def validation_step(self, batch, batch_nb):
        
        images, masks = batch['pixel_values'], batch['labels']
        
        outputs = self(images, masks)
        
        loss, logits = outputs[0], outputs[1]
        
        upsampled_logits = nn.functional.interpolate(
            logits, 
            size=masks.shape[-2:], 
            mode="bilinear", 
            align_corners=False
        )
        
        predicted = upsampled_logits.argmax(dim=1)
        
        self.val_mean_iou.add_batch(
            predictions=predicted.detach().cpu().numpy(), 
            references=masks.detach().cpu().numpy()
        )
        
        return({'val_loss': loss})
    
    def validation_epoch_end(self, outputs):
        metrics = self.val_mean_iou.compute(
              num_labels=self.num_classes, 
              ignore_index=255, 
              reduce_labels=False,
          )
        
        avg_val_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        val_mean_iou = metrics["mean_iou"]
        val_mean_accuracy = metrics["mean_accuracy"]
        
        metrics = {"val_loss": avg_val_loss, "val_mean_iou":val_mean_iou, "val_mean_accuracy":val_mean_accuracy}
        for k,v in metrics.items():
            self.log(k,v)

        return metrics
    
    # def test_step(self, batch, batch_nb):
        
    #     images, masks = batch['pixel_values'], batch['labels']
        
    #     outputs = self(images, masks)
        
    #     loss, logits = outputs[0], outputs[1]
        
    #     upsampled_logits = nn.functional.interpolate(
    #         logits, 
    #         size=masks.shape[-2:], 
    #         mode="bilinear", 
    #         align_corners=False
    #     )
        
    #     predicted = upsampled_logits.argmax(dim=1)
        
    #     self.test_mean_iou.add_batch(
    #         predictions=predicted.detach().cpu().numpy(), 
    #         references=masks.detach().cpu().numpy()
    #     )
            
    #     return({'test_loss': loss})
    
    # def test_epoch_end(self, outputs):
    #     metrics = self.test_mean_iou.compute(
    #           num_labels=self.num_classes, 
    #           ignore_index=255, 
    #           reduce_labels=False,
    #       )
       
    #     avg_test_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
    #     test_mean_iou = metrics["mean_iou"]
    #     test_mean_accuracy = metrics["mean_accuracy"]

    #      metrics = {"test_loss": avg_test_loss, "test_mean_iou":test_mean_iou, "test_mean_accuracy":test_mean_accuracy}
        
    #     for k,v in metrics.items():
    #         self.log(k,v)
        
    #     return metrics
    
    def configure_optimizers(self):
        return torch.optim.Adam([p for p in self.parameters() if p.requires_grad], lr=2e-05, eps=1e-08)
    
    def train_dataloader(self):
        return self.train_dl
    
    def val_dataloader(self):
        return self.val_dl
    
    # def test_dataloader(self):
    #     return self.test_dl

AttributeError: module 'distutils' has no attribute 'version'

In [ ]:
segformer_finetuner = SegformerFinetuner(
    train_dataset.id2label, 
    train_dataloader=train_dataloader, 
    val_dataloader=valid_dataloader, 
    #test_dataloader=test_dataloader, 
    metrics_interval=10,
)

from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint

early_stop_callback = EarlyStopping(
    monitor="val_loss", 
    min_delta=0.00, 
    patience=3, 
    verbose=False, 
    mode="min",
)

checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor="val_loss")

trainer = pl.Trainer(
    gpus=1, 
    callbacks=[early_stop_callback, checkpoint_callback],
    max_epochs=500,
    val_check_interval=len(train_dataloader),
)

trainer.fit(segformer_finetuner)

In [ ]:

'''
# Fine-tuning을 위한 Trainer 및 TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./your_output_directory",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=feature_extractor,
    args=training_args,
    train_dataset=train_dataloader,
    eval_dataset=valid_dataloader,
)

# Fine-tuning 실행
trainer.train()
'''

class TrainerCustom(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)
        logits = outputs.logits
        labels = inputs.get("labels")
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Fine-tuning을 위한 Trainer 및 TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./your_output_directory",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = TrainerCustom(
    model=feature_extractor,
    args=training_args,
    train_dataset=train_dataloader,
    eval_dataset=valid_dataloader,
)

# Fine-tuning 실행
trainer.train()

In [21]:
import torch
#from torch.utils.data import DataLoader
from torchvision import transforms
from transformers import SegformerFeatureExtractor
from transformers import Trainer, TrainingArguments
from dataset import CustomImageDataset

local_data_path = 'C:\Workspace\Personal\ccp_data/Training/image/AP10_City_IMAGE'
server_data_path = '/mnt/nas27_dataset/Dataset/ccp/Training/image/AP10_City_IMAGE/'

# Fine-tuning에 사용할 데이터셋 클래스 초기화
train_dataset = CustomImageDataset(folder_path=local_data_path, transform=None, mode="Train")
valid_dataset = CustomImageDataset(folder_path=local_data_path, transform=None, mode="Valid")


print(len(train_dataset))
print(len(valid_dataset))

#for i in range(min(5, len(train_dataset))):
    #sample = train_dataset[i]
    #print(sample)



C:\Workspace\Personal\ccp/Training/image/AP10_City_IMAGE Done.
C:\Workspace\Personal\ccp/Training/image/AP10_City_SH Done.
C:\Workspace\Personal\ccp/Training/label/AP10_City_Carbon Done.
C:\Workspace\Personal\ccp/Training/label/AP10_City_GT Done.
C:\Workspace\Personal\ccp/Validation/image/AP10_City_IMAGE Done.
C:\Workspace\Personal\ccp/Validation/image/AP10_City_SH Done.
C:\Workspace\Personal\ccp/Validation/label/AP10_City_Carbon Done.
C:\Workspace\Personal\ccp/Validation/label/AP10_City_GT Done.
3200
400
